For run this code, you must be installed the POSEIDON package and the POSEIDON data required.  Please follow the instructions in the [POSEIDON documentation](https://poseidon-retrievals.readthedocs.io/en/latest/content/installation.html).

In [ ]:
from POSEIDON.core import create_star
from POSEIDON.constants import R_Sun
from POSEIDON.stellar import stellar_contamination
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Define stellar properties
R_s = 0.1192 * R_Sun  # Stellar radius (m)
T_s = 2566            # Stellar effective temperature (K)
Met_s = 0.05          # Stellar metallicity
log_g_s = 5.2396      # Stellar surface gravity (log g)

# Spot and facula temperatures
T_spot = 0.86 * T_s   # Spot temperature
T_fac = T_s + 100     # Facula temperature

# Coverage fractions (min, central, max)
f_spot_values = [0.01, 0.08, 0.26]
f_fac_values = [0.08, 0.54, 0.70]

# Load wavelength data
wl = np.loadtxt("waves.txt")

# Generate all combinations of f_spot and f_fac
combinations = list(itertools.product(f_spot_values, f_fac_values))

epsilon_list = []
labels = []

# Calculate stellar contamination for each combination
for f_spot, f_fac in combinations:
    star = create_star(
        R_s, T_s, log_g_s, Met_s,
        stellar_grid="phoenix",
        stellar_contam="two_spots",
        f_spot=f_spot, T_spot=T_spot,
        f_fac=f_fac, T_fac=T_fac,
    )
    epsilon = stellar_contamination(star, wl)
    epsilon_list.append(epsilon)

    filename = f"TRAPPIST-1_contam_fspot{f_spot}_ffac{f_fac}.txt"
    np.savetxt(filename, np.column_stack((wl, epsilon)))

    labels.append(f"f_spot={f_spot}, f_fac={f_fac}")

# Plot the stellar contaminations
plt.figure(figsize=(12, 8))
for epsilon, label in zip(epsilon_list, labels):
    plt.plot(wl, epsilon, label=label)

plt.xlabel("Longitud de onda (μm)")
plt.ylabel("Contaminación estelar ε")
plt.title("Contaminación estelar para diferentes fracciones de cobertura")
plt.legend()
plt.show()